In [1]:
from lxml import html  
import csv,os,json
import requests
import pandas as pd
from time import sleep
import socket
import socks

In [9]:
socks.setdefaultproxy(proxy_type=socks.PROXY_TYPE_SOCKS5, addr="127.0.0.1", port=9050)
socket.socket = socks.socksocket
print(requests.get("http://icanhazip.com").text)

185.217.93.22



In [10]:
def AmzonParser(url):
    headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.90 Safari/537.36'}
    page = requests.get(url,headers=headers)
    while True:
        sleep(3)
        try:
            doc = html.fromstring(page.content)
            XPATH_NAME = '//h1[@id="title"]//text()'
            #XPATH_SALE_PRICE = '//span[contains(@id,"ourprice") or contains(@id,"saleprice")]/text()'
            XPATH_SALE_PRICE = '//span[contains(@class,"offer-price") and contains(@class,"a-size-medium")]//text()'
            #XPATH_ORIGINAL_PRICE = '//td[contains(text(),"List Price") or contains(text(),"M.R.P") or contains(text(),"Price")]/following-sibling::td/text()'
            XPATH_CATEGORY = '//a[@class="a-link-normal a-color-tertiary"]//text()'
            XPATH_AVAILABILITY = '//div[@id="availability"]//text()' 
            XPATH_AUTHOR = '//a[contains(@class,"contributorNameID")]//text()'
            XPATH_ISBN10 = '//li/b[contains(text(),"ISBN-10:")]/following-sibling::text()'
            XPATH_PAGES = '//li/b[contains(text(),"Paperback")]/following-sibling::text()'
            XPATH_PUBLISHER = '//li/b[contains(text(),"Publisher")]/following-sibling::text()'
            XPATH_LANGUAGE = '//li/b[contains(text(),"Language")]/following-sibling::text()'
            XPATH_DIMENSIONS = '//li/b[contains(text(),"Dimensions")]/following-sibling::text()'
            XPATH_WEIGHT = '//li/b[contains(text(),"Weight")]/following-sibling::text()'
            XPATH_RATING = '//li/b[contains(text(),"Average Customer Review:")]/following-sibling::span/span/a/i/span/text()'
           
            RAW_NAME = doc.xpath(XPATH_NAME)
            RAW_SALE_PRICE = doc.xpath(XPATH_SALE_PRICE)
            RAW_CATEGORY = doc.xpath(XPATH_CATEGORY)
            ##RAW_ORIGINAL_PRICE = doc.xpath(XPATH_ORIGINAL_PRICE)
            RAw_AVAILABILITY = doc.xpath(XPATH_AVAILABILITY)
            RAW_AUTHOR = doc.xpath(XPATH_AUTHOR)
            RAW_ISBN10 = doc.xpath(XPATH_ISBN10)
            RAW_PAGES = doc.xpath(XPATH_PAGES)
            RAW_PUBLISHER = doc.xpath(XPATH_PUBLISHER) 
            RAW_LANGUAGE = doc.xpath(XPATH_LANGUAGE)
            RAW_DIMENSIONS = doc.xpath(XPATH_DIMENSIONS)
            RAW_WEIGHT = doc.xpath(XPATH_WEIGHT)
            RAW_RATING = doc.xpath(XPATH_RATING)
            
            print(RAW_RATING)
 
            NAME = ' '.join(''.join(RAW_NAME).split()) if RAW_NAME else None
            SALE_PRICE = ' '.join(''.join(RAW_SALE_PRICE).split()).strip() if RAW_SALE_PRICE else None
            CATEGORY = ' > '.join([i.strip() for i in RAW_CATEGORY]) if RAW_CATEGORY else None
            #ORIGINAL_PRICE = ''.join(RAW_ORIGINAL_PRICE).strip() if RAW_ORIGINAL_PRICE else None
            AVAILABILITY = ''.join(RAw_AVAILABILITY).strip() if RAw_AVAILABILITY else None
            AUTHOR = ''.join(RAW_AUTHOR).strip() if RAW_AUTHOR else None
            ISBN10 = ''.join(RAW_ISBN10).strip() if RAW_ISBN10 else None
            PAGES = ''.join(RAW_PAGES).strip() if RAW_PAGES else None
            PUBLISHER = ''.join(RAW_PUBLISHER).strip() if RAW_PUBLISHER else None
            LANGUAGE = ''.join(RAW_LANGUAGE).strip() if RAW_LANGUAGE else None
            DIMENSIONS = ''.join(RAW_DIMENSIONS).strip() if RAW_DIMENSIONS else None
            WEIGHT = ''.join(RAW_WEIGHT).strip().strip('()').strip() if RAW_WEIGHT else None
            RATING = ''.join(RAW_RATING).strip() if RAW_RATING else None
            #if not ORIGINAL_PRICE:
                #ORIGINAL_PRICE = SALE_PRICE
 
            if page.status_code!=200:
                raise ValueError('captha')
            data = {
                    'NAME':NAME,
                    'SALE_PRICE':SALE_PRICE,
                    'CATEGORY':CATEGORY,
                    #'ORIGINAL_PRICE':ORIGINAL_PRICE,
                    'AVAILABILITY':AVAILABILITY,
                    'URL':url,
                    'AUTHOR' : AUTHOR,
                    'ISBN10' : ISBN10,
                    'PAGES' : PAGES,
                    'PUBLISHER' : PUBLISHER,
                    'LANGUAGE' : LANGUAGE,
                    'DIMENSIONS' : DIMENSIONS,
                    'WEIGHT' : WEIGHT,
                    'RATING' : RATING,
                    }
 
            return data
        except Exception as e:
            print(e)
 

In [11]:
def ReadAsin(AsinList):
    # AsinList = csv.DictReader(open(os.path.join(os.path.dirname(__file__),"Asinfeed.csv")))
    extracted_data = []
    for i in AsinList:
        url = "http://www.amazon.com/gp/product/"+i
        print("Processing: "+url)
        extracted_data.append(AmzonParser(url))
        sleep(5)
    f=open('data.json','w')
    json.dump(extracted_data,f,indent=4)
 

In [12]:
formatted_asin = []
df = pd.read_csv("amazon_product_asins.csv")
for index, row in df.iterrows():
    if len(row[0]) < 8:
        asin = "000" + row[0]
        formatted_asin.append(asin)
    elif len(row[0]) < 9:
        asin = "00" + row[0]
        formatted_asin.append(asin)
    elif len(row[0]) < 10:
        asin = "0" + row[0]
        formatted_asin.append(asin)
    else:
        formatted_asin.append(row[0])

In [13]:
if __name__ == "__main__":
    ReadAsin(formatted_asin[500:700])

Processing: http://www.amazon.com/gp/product/0451163966
['4.3 out of 5 stars']
Processing: http://www.amazon.com/gp/product/0393327345
['4.4 out of 5 stars']
Processing: http://www.amazon.com/gp/product/0399167501
['4.4 out of 5 stars']
Processing: http://www.amazon.com/gp/product/0143034901
['4.5 out of 5 stars']
Processing: http://www.amazon.com/gp/product/1586638459
['4.5 out of 5 stars']
Processing: http://www.amazon.com/gp/product/0143132172
['3.2 out of 5 stars']
Processing: http://www.amazon.com/gp/product/0062419013
['4.4 out of 5 stars']
Processing: http://www.amazon.com/gp/product/0399592881
['4.0 out of 5 stars']
Processing: http://www.amazon.com/gp/product/0393089053
['4.7 out of 5 stars']
Processing: http://www.amazon.com/gp/product/1250105625
['4.3 out of 5 stars']
Processing: http://www.amazon.com/gp/product/1524797022
['4.4 out of 5 stars']
Processing: http://www.amazon.com/gp/product/0425247449
['4.4 out of 5 stars']
Processing: http://www.amazon.com/gp/product/0786965

Processing: http://www.amazon.com/gp/product/1250122996
[]
Processing: http://www.amazon.com/gp/product/1501143484
['4.5 out of 5 stars']
Processing: http://www.amazon.com/gp/product/1480030473
['4.5 out of 5 stars']
Processing: http://www.amazon.com/gp/product/0307477479
['3.6 out of 5 stars']
Processing: http://www.amazon.com/gp/product/0451476204
[]
Processing: http://www.amazon.com/gp/product/0735219192
[]
Processing: http://www.amazon.com/gp/product/1400078776
['3.7 out of 5 stars']
Processing: http://www.amazon.com/gp/product/150116340X
['4.0 out of 5 stars']
Processing: http://www.amazon.com/gp/product/1616149981
['4.4 out of 5 stars']
Processing: http://www.amazon.com/gp/product/052595497X
['4.5 out of 5 stars']
Processing: http://www.amazon.com/gp/product/0394800877
['4.8 out of 5 stars']
Processing: http://www.amazon.com/gp/product/0399593489
['4.3 out of 5 stars']
Processing: http://www.amazon.com/gp/product/0525509712
['4.3 out of 5 stars']
Processing: http://www.amazon.com

In [67]:
a = "bbb"
a.strip().strip(')').strip('()').strip()

'bbb'

In [58]:
a

'bbb () '